In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import scipy.io as sio  # for loading matfile
import numpy as np
import cv2
import random
import glob

## Data Preprocessing

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/car_ims.tgz \
  -O /tmp/car_ims.tgz

--2020-07-02 13:09:13--  http://imagenet.stanford.edu/internal/car196/car_ims.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘/tmp/car_ims.tgz’

/tmp/car_ims.tgz    100%[===================>]   1.82G  8.02MB/s    in 2m 35s  

2020-07-02 13:11:49 (12.0 MB/s) - ‘/tmp/car_ims.tgz’ saved [1956628579/1956628579]



In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_annos.mat \
  -O /tmp/cars_annos.mat


--2020-07-02 13:12:33--  http://imagenet.stanford.edu/internal/car196/cars_annos.mat
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394471 (385K) [text/plain]
Saving to: ‘/tmp/cars_annos.mat’

/tmp/cars_annos.mat 100%[===================>] 385.23K   707KB/s    in 0.5s    

2020-07-02 13:12:34 (707 KB/s) - ‘/tmp/cars_annos.mat’ saved [394471/394471]



In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat \
  -O /tmp/cars_annos.mat


--2020-07-02 13:32:08--  http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185758 (181K) [text/plain]
Saving to: ‘/tmp/cars_annos.mat’

/tmp/cars_annos.mat 100%[===================>] 181.40K   456KB/s    in 0.4s    

2020-07-02 13:32:08 (456 KB/s) - ‘/tmp/cars_annos.mat’ saved [185758/185758]



In [ ]:
import os
import tarfile

local_tar = '/tmp/car_ims.tgz'

my_tar = tarfile.open(local_tar)
my_tar.extractall('/tmp')
my_tar.close()

In [ ]:
def save_train_data(fnames, labels,bboxes):
    src_folder ='/tmp/car_ims/'
    num_samples = len(fnames)

    train_split = 0.8
    num_train = int(round(num_samples * train_split))
    train_indexes = random.sample(range(num_samples), num_train)

    for i in range(num_samples):
        fname = fnames[i]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)
        src_image = cv2.imread(src_path)
        height, width = src_image.shape[:2]

        # margins of 16 pixels (Crop image according to a given bounding box)
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)

        if i in train_indexes:
            dst_folder = '/tmp/car_ims/train/'
        else:
            dst_folder = '/tmp/car_ims/valid/'

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)
        dst_image = src_image[y1:y2, x1:x2]

        cv2.imwrite(dst_path, dst_image)

In [ ]:
car_annotation= sio.loadmat('/tmp/cars_annos.mat')

annotations = car_annotation['annotations']
annotations = np.transpose(annotations)

fnames = []
class_ids = []
labels = []
bboxes  = []

for annotation in annotations:
  bbox_x1 = annotation[0][0][0][0]
  bbox_y1 = annotation[0][1][0][0]
  bbox_x2 = annotation[0][2][0][0]
  bbox_y2 = annotation[0][3][0][0]

  class_id = annotation[0][4][0][0]
  bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
  labels.append(f'{class_id:06}')

  file_name = str(0)+annotation[0][5][0]
  class_ids.append(class_id)
  fnames.append(file_name)

labels_count = np.unique(class_ids).shape[0]
print(np.unique(class_ids))
print('The number of different cars is %d' % labels_count)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196]
The number of different cars is 196


In [ ]:
save_train_data(fnames, labels, bboxes)

In [ ]:
print(glob.glob("/tmp/car_ims/train/*"))
print(glob.glob("/tmp/car_ims/valid/*"))

['/tmp/car_ims/train/000095', '/tmp/car_ims/train/000145', '/tmp/car_ims/train/000016', '/tmp/car_ims/train/000178', '/tmp/car_ims/train/000015', '/tmp/car_ims/train/000127', '/tmp/car_ims/train/000162', '/tmp/car_ims/train/000116', '/tmp/car_ims/train/000152', '/tmp/car_ims/train/000113', '/tmp/car_ims/train/000041', '/tmp/car_ims/train/000112', '/tmp/car_ims/train/000173', '/tmp/car_ims/train/000058', '/tmp/car_ims/train/000085', '/tmp/car_ims/train/000005', '/tmp/car_ims/train/000011', '/tmp/car_ims/train/000034', '/tmp/car_ims/train/000018', '/tmp/car_ims/train/000060', '/tmp/car_ims/train/000102', '/tmp/car_ims/train/000177', '/tmp/car_ims/train/000156', '/tmp/car_ims/train/000068', '/tmp/car_ims/train/000135', '/tmp/car_ims/train/000062', '/tmp/car_ims/train/000129', '/tmp/car_ims/train/000009', '/tmp/car_ims/train/000196', '/tmp/car_ims/train/000128', '/tmp/car_ims/train/000169', '/tmp/car_ims/train/000161', '/tmp/car_ims/train/000027', '/tmp/car_ims/train/000061', '/tmp/car_ims